In [1]:
import pandas as pd
from glob import glob

In [2]:
# import Lending Club data as a DataFrame
lc_2007_2015 = pd.read_csv('data/lending_club_data_unzip/lending_club_loans_2007_2015.csv', header=0, low_memory=False)#, skiprows=[0])

In [3]:
# inspect columns and respective types
lc_2007_2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887379 entries, 0 to 887378
Data columns (total 74 columns):
id                             887379 non-null int64
member_id                      887379 non-null int64
loan_amnt                      887379 non-null float64
funded_amnt                    887379 non-null float64
funded_amnt_inv                887379 non-null float64
term                           887379 non-null object
int_rate                       887379 non-null float64
installment                    887379 non-null float64
grade                          887379 non-null object
sub_grade                      887379 non-null object
emp_title                      835917 non-null object
emp_length                     842554 non-null object
home_ownership                 887379 non-null object
annual_inc                     887375 non-null float64
verification_status            887379 non-null object
issue_d                        887379 non-null object
loan_status          

In [4]:
# define functions to clean the data and format columns for a classification model

In [5]:
def relevant_cols(X):
    
    """ return dataFrame containing only useful and relvant columns """
    
    # select relevant columns
    relevant_cols = ['loan_amnt','funded_amnt','term',
                 'int_rate','installment','grade',
                 'emp_length','home_ownership',
                 'annual_inc','verification_status',
                 'loan_status','pymnt_plan','purpose',
                 'dti','delinq_2yrs', 'inq_last_6mths',
                 'open_acc', 'total_rec_late_fee',
                 'pub_rec', 'revol_bal', 'total_acc', 
                 'out_prncp', 'total_pymnt', 
                 'total_rec_prncp', 'total_rec_int', 
                 'recoveries', 'last_pymnt_amnt',  
                 'application_type', 'acc_now_delinq', 
                 'tot_coll_amt', 'tot_cur_bal']
   
    # filter columns and return
    return X[relevant_cols]

In [6]:
def term_get_dums(X):

    """Create binary categories for term, 1=60mnthTerm, 0=30mnthTerm and return the updated DataFrame"""
    
    # select term column
    term = X[['term']]
    
    # get dummies for term column
    term = pd.get_dummies(term)
 
    # replace term with binary features
    X.term = term['term_ 60 months']
    
    # return updated DataFrame
    return X


In [7]:
def grade_get_dums(X):
    
    """ Create dummies for grades A to G, return df without grades and grade_G """
    
    # select grades column
    grade = X[['grade']]
    
    # get dummies for each grade
    grade = pd.get_dummies(grade)
    
    # drop redundant column
    if 'grade_G' in grade.columns:
        grade = grade.iloc[:, :-1]
    
    # append new columns to X
    X[grade.columns] = grade
    
    # drop column being replaced by dummy columns
    X = X.drop('grade', axis=1)
    
    # return updated DataFrame
    return X

In [8]:
def convert_emp_length(X):
    
    """ convert emp_length to int number of years"""
    
    # select employment length column
    emp_length = X[['emp_length']]
    
    # define lenght of employment dict
    len_dict = {'< 1 year':0,
                '1 year':1,
                '2 years':2,
                '3 years':3,
                '4 years':4,
                '5 years':5,
                '6 years':6,
                '7 years':7,
                '8 years':8,
                '9 years':9,
                '10+ years':10,
                'empty': -1
                }
    
    # replace missing values
    emp_length.fillna('empty')
    
    # map length of employment to dataFrame
    emp_length.emp_length = emp_length.emp_length.map(len_dict)
    
    # replace column
    X.emp_length = emp_length.emp_length
    
    # return updated DataFrame
    return X
    
    
    

In [9]:
def home_ownership_get_dums(X):
    
    """ Create dummies for home ownership, return the updated DataFrame"""
    
    # select home ownership column
    home = X[['home_ownership']]
    
    # create dummy variables
    home = pd.get_dummies(home)
    
    # discard redundant columns
    home = home[['home_ownership_OWN', 'home_ownership_RENT','home_ownership_MORTGAGE']]
    
    # drop original column from original DataFrame
    X = X.drop('home_ownership', axis=1)
    
    # append new columns 
    X[home.columns] = home
    
    # return updated dataFrame
    return X
    
    
    

In [10]:
def verification_get_dums(X):
    
    """ Create dummies for loan verification status, return the updated DataFrame"""
    
    # select verification column
    verification = X[['verification_status']]
    
    # get dummies
    verification = pd.get_dummies(verification)
    
    verification = verification.drop('verification_status_Not Verified', axis=1)
    
    
    X[verification.columns] = verification
    
    X = X.drop('verification_status', axis=1)
    
    return X

In [11]:
def convert_pymnt_plan(X):
    
    """ create dummy var for payment plan column and return the updated DataFrame """
    
    plan_dict = {
        'y': 1,
        'n': 0
    }
    
    X.pymnt_plan = X.pymnt_plan.map(plan_dict)
    
    return X
    

In [12]:
def convert_application_type(X):
    
    """Convert application_type column to binary joint application column"""
    
    # create boolean joint application column
    X['joint_application'] = (X.application_type=='JOINT')*1
    
    # drop original column
    X = X.drop('application_type', axis=1)
    
    # return updated DataFrame
    return X

    

In [13]:
def convert_percentage(X):
    
    """ Convert precentage from object to float if necessary"""
    
    # check type and convert to float if object
    lambda_convert = lambda percent : percent if (type(percent) != str) else float(percent[1:-1])/100
    
    # apply lambda function to column
    X.int_rate = X.int_rate.apply(lambda_convert)
    
    return X
    
    

In [14]:
def purpose_get_dums(X):
    
    """ Create binary features for loan purpose and return updated DataFrame"""
    
    # select purpose column
    purpose = X[['purpose']]
    
    # create dummy variables
    purpose = pd.get_dummies(purpose)
    
    # drop redundant columns
    purpose = purpose.drop('purpose_other', axis=1)
    
    # append new columns
    X[purpose.columns] = purpose
    
    # drop the original column
    X = X.drop('purpose', axis=1)
    
    #return the updated DataFrame
    return X
    

In [15]:
def convert_loan_status(X):
    
    """ Create binary feature for loan status, 1=Default """
    
    # compute boolean binary feature
    X['default'] = (X.loan_status=='Default')*1
    
    # drop original column
    X = X.drop('loan_status', axis=1)
    
    # return updated DataFrame
    return X

In [16]:
# define function to execute data cleaning and wrangling

In [17]:
def data_pipeline(raw):
    
    """Data Pipeline function to clean the raw data"""
    
    # remove irrelevant columns
    X = relevant_cols(raw)
    
    # get binary features for term
    X = term_get_dums(X)
    
    # get binary features for grade
    X = grade_get_dums(X)
    
    # convert length of employment to int
    X = convert_emp_length(X)
    
    # get binary features for home ownership
    X = home_ownership_get_dums(X)
    
    # get binary features for verification
    X = verification_get_dums(X)
    
    # convert payment plan to binary feature
    X = convert_pymnt_plan(X)
    
    # create binary feature for joint applications
    X = convert_application_type(X)
    
    # create binary features for loan purpose
    X = purpose_get_dums(X)
    
    # create binary features for loan status
    X = convert_loan_status(X)
    
    # convert type for percentages as necessary
    X = convert_percentage(X)
    
    
    return X
    
    
    

In [18]:
def process_files(first_df):
    # collect filenames with glob
    filenames = glob('data/lending_club_data_unzipped/LoanStats*')
    
    all_dfs = first_df
    
    # avoid list comprehension due to memory error
    for f in filenames[::]:
        
        # read in single csv file as dataframe
        print('reading file: ', f)
        X = pd.read_csv(f, header=0, low_memory=False, skiprows=[0])
        
        # feed data through cleaning pipeline
        X = data_pipeline(X)
        
        # drop rows with any missing data
        X = X.dropna()
        
        # concat/append dataframe 
        all_dfs = pd.concat([all_dfs, X], sort=False)
        
        print(all_dfs.shape)
        
    # fill additional columns with 0
    cols_to_fill = ['grade_E', 'grade_F', 'purpose_educational', 'purpose_wedding', 'tot_coll_amt', 'tot_cur_bal', 'emp_length']
    all_dfs[cols_to_fill] = all_dfs[cols_to_fill].fillna(0)

        
    return all_dfs


In [19]:
### Clean the data

X = data_pipeline(lc_2007_2015)
#X = X.dropna()

# inspect final columns
X.info()

/home/alhalarewicz/anaconda3/envs/SBEnv/lib/python3.7/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/home/alhalarewicz/anaconda3/envs/SBEnv/lib/python3.7/site-packages/pandas/core/frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887379 entries, 0 to 887378
Data columns (total 51 columns):
loan_amnt                              887379 non-null float64
funded_amnt                            887379 non-null float64
term                                   887379 non-null uint8
int_rate                               887379 non-null float64
installment                            887379 non-null float64
emp_length                             842554 non-null float64
annual_inc                             887375 non-null float64
pymnt_plan                             887379 non-null int64
dti                                    887379 non-null float64
delinq_2yrs                            887350 non-null float64
inq_last_6mths                         887350 non-null float64
open_acc                               887350 non-null float64
total_rec_late_fee                     887379 non-null float64
pub_rec                                887350 non-null float64
revol_bal  

In [20]:
result = process_files(X)
result.shape

(887379, 51)

In [21]:
# write student loan data to file
print('writing to file: student_loans.csv')
result[result.purpose_educational==1].to_csv('data/clean_data/student_loans.csv')

# write other loan data to file
print('writing to file: other_loans.csv')
result[result.purpose_educational==0].to_csv('data/clean_data/other_loans.csv')
print('DONE')

writing to file: student_loans.csv
writing to file: other_loans.csv
DONE


In [22]:
result = result[result.purpose_educational==1]
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 423 entries, 25230 to 790101
Data columns (total 51 columns):
loan_amnt                              423 non-null float64
funded_amnt                            423 non-null float64
term                                   423 non-null uint8
int_rate                               423 non-null float64
installment                            423 non-null float64
emp_length                             423 non-null float64
annual_inc                             423 non-null float64
pymnt_plan                             423 non-null int64
dti                                    423 non-null float64
delinq_2yrs                            423 non-null float64
inq_last_6mths                         423 non-null float64
open_acc                               423 non-null float64
total_rec_late_fee                     423 non-null float64
pub_rec                                423 non-null float64
revol_bal                              423 non-null 